In [1]:
!pip install stumpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.1/169.1 kB 3.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
from tqdm import tqdm
import csv
import sqlalchemy
import stumpy
import numpy as np
from collections import Counter
import csv
import os
import psycopg2
from datetime import datetime, timedelta
pd.set_option('display.width', 200)

# Functions

In [3]:


def get_stocks_list(file_path,rn):
    """
    Retrieves a list of stock names from a CSV file.

    Args:
    - file_path (str): Path to the CSV file.
    - rn (int): Index of the column containing stock names.

    Returns:
    - stocks_list (list): List of stock names.
    """
    stocks_list = []
    with open(file_path, 'r', newline='') as csvfile:
        csv_reader = csv.reader(csvfile)
        next(csv_reader)  # Skip the header row
        for row in csv_reader:
            stocks_list.append(row[rn])
    return stocks_list

# Queary Pattern....................................
def Query_pattern_length(window_time):
    """
    Calculates the query pattern length based on the specified window time.

    Args:
    - window_time (int): The window time.

    Returns:
    - query_pattern_length (int): The calculated query pattern length.
    """
    if window_time ==15:
        query_pattern_length = window_time * 3
    elif window_time ==30:
        query_pattern_length = window_time * 2
    elif window_time == 60:
        query_pattern_length = window_time * 1.5
    elif window_time ==120:
        query_pattern_length=(window_time/5) * 3
    elif window_time in [180,240]:
        query_pattern_length=(window_time/5) * 2.5
    elif window_time in [360,375]:
        query_pattern_length=(window_time/5) * 1.5
    else:
        raise ValueError("Window time must be 15, 30, 60, 120, 180, 240, 360, 375")

    return int(query_pattern_length)

def query_pattern(raw_data, stock, query_pattern_length, window_time):
    # Assign the raw_data to a new variable time_series
    time_series = raw_data

    # Check if window_time is greater than or equal to 120
    if window_time >= 120:
        # Resample the time_series to every 5th row and reverse the order
        time_series = time_series.iloc[::-5]
        time_series = time_series.iloc[::-1]
        print(time_series)
        # Reset the index of the DataFrame
        time_series = time_series.reset_index()

        # Find the index and raw_index of the row corresponding to the specified stock and date
        index, raw_index = (
            time_series[(time_series['Stock'] == stock) ].index[-1],
            time_series[(time_series['Stock'] == stock) ]['index'].iloc[-1]
        )
        raw_index = raw_index  # - 4

    else:

        # Reset the index of the DataFrame
        time_series = time_series.reset_index()
        # Find the index of the row corresponding to the specified stock and date
        index = time_series[(time_series['Stock'] == stock)].index[-1]
        raw_index = index


    # Calculate the start and end index for selecting the pattern
    selected_row_index = index
    start_index = (1 - query_pattern_length) + selected_row_index
    end_index = selected_row_index + 1

    # If window_time is less than 120, drop the 'index' column from the DataFrame
    if window_time < 120:
        dat = time_series.drop(columns=['index'])
        # Select the pattern from the DataFrame using the start and end index
        selected_pattern = dat[start_index:end_index]
    else:
        # Select the pattern from the DataFrame using the start and end index
        selected_pattern = time_series[start_index:end_index]

    # Increment the raw_index by 1
    raw_index = raw_index + 1

    # Return the index, raw_index, selected_pattern, time_series, and raw_index
    return index, raw_index, selected_pattern, time_series, raw_index

#cd = query_pattern(combined_data,'ITC','2022-03-09',72,120)

def find_matches(selected_pattern, time_series, query_pattern_length, valid_match, col_name):
    # Find matches using the STUMPY library
    matches = stumpy.match(selected_pattern[col_name], time_series[col_name], normalize=True, max_distance=np.inf, max_matches=40)

    if valid_match:  # If only valid matches are required
        valid_matches = []
        for match in matches[1:]:  # Skip the first match since it's an exact match with itself
            ed, idx = match  # Extract Euclidean distance and index
            st = time_series.iloc[idx]["Stock"]  # Get the stock value at the start index of the match
            en = time_series.iloc[idx + (query_pattern_length - 1)]["Stock"]  # Get the stock value at the end index of the match
            if st == en:  # Check if the start and end stock values are the same
                new_row = [ed, idx]  # Create a new row with Euclidean distance and index
                valid_matches.append(new_row)  # Add the row to valid matches
        return pd.DataFrame(valid_matches)  # Return DataFrame of valid matches
    return pd.DataFrame(matches)  # Return DataFrame of all matches if valid_match is False

#mat = find_matches(selected_pattern,time_series,72,True,'Close')


def subsequent_df(time_series, duration_in_minutes, matches, window_time, time_series_mod):
    """
    Generate subsequent DataFrames for each match in the time series.

    Args:
    - time_series (DataFrame): The original time series DataFrame.
    - duration_in_minutes (int): Duration of each match in minutes.
    - matches (tuple): Tuple containing arrays of Euclidean distances and match indices.
    - window_time (int): Window size for subsequent DataFrame in minutes.
    - time_series_mod (DataFrame): Modified time series DataFrame.

    Returns:
    - all_matches_dfs (list): List of DataFrames containing matched periods.
    - all_subsequent_dfs (list): List of DataFrames containing subsequent periods.
    """
    all_matches_dfs = []
    all_subsequent_dfs = []
    match_number = 0

    # Iterate through each match
    for ed, idx in zip(matches[0], matches[1]):

        if window_time >= 120:
            # Adjust index for window_time >= 120
            idx = time_series_mod.iloc[idx]['index'] - 4

        match_number += 1

        # Define end index for the matched period
        end_idx = idx + duration_in_minutes

        # Create DataFrame for the matched period
        match_df = time_series.iloc[idx:end_idx].copy()
        match_df['Match_Number'] = match_number

        # Define start and end indices for the subsequent period
        subsequent_start_index = end_idx
        subsequent_end_index = subsequent_start_index + window_time

        # Create DataFrame for the subsequent period
        subsequent_df = time_series.iloc[subsequent_start_index:subsequent_end_index].copy()
        subsequent_df['Match_Number'] = match_number

        # Check for valid subsequent_df
        try:
            if match_df.iloc[-1]['Stock'] == subsequent_df.iloc[-1]['Stock']:
                all_matches_dfs.append(match_df)
                all_subsequent_dfs.append(subsequent_df)

                with open('match.csv', 'a', newline='') as csvfile:

                    csv_writer = csv.writer(csvfile)
                    column_names = match_df.columns.tolist()
                    csv_writer.writerows([column_names])
                    csv_writer.writerows(match_df.values.tolist())

                with open('subsequent.csv', 'a', newline='') as csvfile:

                    csv_writer = csv.writer(csvfile)
                    column_names = subsequent_df.columns.tolist()
                    csv_writer.writerows([column_names])
                    csv_writer.writerows(subsequent_df.values.tolist())


            else:
                # Subtract 1 for invalid subsequent_df
                match_number = match_number - 1
                print("Invalid Match detected")
        except Exception as e:
            print(str(e))
            match_number = match_number - 1
            print('Subsequent Period Not Available for This Interval [index {} {}]'.format(subsequent_start_index, subsequent_end_index))

    return all_matches_dfs, all_subsequent_dfs


def buyorsell(matches_dfs, subsequent_dfs,threshold):
    """
    Determines whether to buy, sell, or remain indecisive based on risk-reward ratio.

    Args:
    - matches_dfs (list): List of DataFrames containing matches.
    - subsequent_dfs (list): List of DataFrames containing subsequent data points.

    Returns:
    - max_action (str): The recommended trading action.
    """
    global c
    bs_index = {'Buy': [], 'Sell': []}
    action_counts = Counter()
    for index, (match, subsequent) in enumerate(zip(matches_dfs, subsequent_dfs)):
        # Out of range
        try:
          close_price = match['Close'].iloc[-1]
          highest_high = subsequent['High'].max()
          lowest_low = subsequent['Low'].min()
        except:
          print(subsequent)
          continue

        buy = highest_high - close_price
        sell = close_price - lowest_low

        reward = max(buy,sell)
        risk = min(buy,sell)

        if risk != 0:
            # Calculate risk-reward ratio
            risk_reward_ratio = abs(reward / risk)
        else:
            risk_reward_ratio = float('inf')



        if risk_reward_ratio > threshold:
            action = 'Buy' if buy >= sell else 'Sell'

            bs_index[action].append(subsequent.reset_index(drop=True)['Match_Number'][0])
        else:
            action = 'Indecision'

        # Update action counts
        action_counts[action] += 1


    return action_counts,bs_index #max_action, action_counts,bs_index


def append_to_csv(filename, data, column_names=None):
    """
    Append data to a CSV file.

    Parameters:
    - filename: The name of the CSV file to append to.
    - data: The data to append to the CSV file. This should be a list of rows, where each row is a list of values.
    - column_names: Optional. The column names for the CSV file. If provided, they will be written as the first row.
    """
    # Check if the file already exists
    file_exists = os.path.exists(filename)

    # Open the CSV file in append mode
    with open(filename, 'a', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)

        # Write column names if provided and the file does not exist
        if column_names and not file_exists:
            csv_writer.writerow(column_names)

        # Append data rows to the CSV file
        for row in data:
            csv_writer.writerow(row)


def majority_decision(action_counts, decision_threshold):
    '''
    Determines the majority decision based on action counts and a decision_threshold.

    Args:
    - action_counts: A dictionary containing counts of 'Buy', 'Sell', and 'Indecision'.
    - decision_threshold: The minimum count required for an action to be considered the majority.

    Returns:
    - The majority decision ('Buy', 'Sell', or 'Indecision').
    '''

    decision_threshold = int(np.ceil(decision_threshold))

    # Get the counts of 'Buy', 'Sell', and 'Indecision' from the action_counts dictionary
    buy_count = action_counts.get('Buy', 0)
    sell_count = action_counts.get('Sell', 0)
    indecision_count = action_counts.get('Indecision', 0)

    # Determine the majority decision based on counts and the provided threshold
    if buy_count > decision_threshold and buy_count > sell_count and buy_count > indecision_count:
        max_action = 'Buy'
    elif sell_count > decision_threshold and sell_count > buy_count and sell_count > indecision_count:
        max_action = 'Sell'
    else:
        max_action = 'Indecision'

    return max_action
def data_start_last_date(stock):
  dbname = "tsdb"#"finflash"
  user = "tsdbadmin"#"suresh"
  password = "dsymdggikni2q119"#"s1u2r3e4"
  host = "voipsetkl8.mzph8npyy2.tsdb.cloud.timescale.com"#"postgresql-152351-0.cloudclusters.net"
  port = "31768"#"19991"
  combined_data = pd.DataFrame()

  conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
  cur = conn.cursor()
  query = '''SELECT * FROM nse_stocck_1min_cleaned Where "Stock"=%s ORDER BY "Date" DESC, "Time" DESC LIMIT 1;'''
  cur.execute(query, (stock,))
  df = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
  date_string = pd.concat([combined_data, df], ignore_index=True)['Date'][0]
  conn.close()
  print(date_string)

  date_object = datetime.strptime(date_string, "%Y-%m-%d").date()

  # Calculate the date three years ago
  end_date = date_string
  start_date  = str(date_object - timedelta(days=365 * 3))

  print(type(start_date),type(end_date))

  return start_date,end_date

def get_same_industry_list(industryStocks, stock):
    try:
      # Find the industry of the specified stock
      industry_name = industryStocks[industryStocks['Ticker'] == stock]['Industry'].values[0]
      # Filter stocks based on the same industry
      filtered_stocks = industryStocks[industryStocks['Industry'] == industry_name]
      # Extract list of stocks belonging to the same industry
      same_industry_stock_list = filtered_stocks['Ticker'].tolist()
      return same_industry_stock_list
    except:
      return None

def get_stock_data(stocks_list,start_date,end_date):

    """
    Retrieves stock data from a database.

    Args:
    - stocks_list (list): List of stock names.

    Returns:
    - combined_data (pd.DataFrame): Combined stock data from the database.
    """
    dbname = "tsdb"#"finflash"
    user = "tsdbadmin"#"suresh"
    password = "dsymdggikni2q119"#"s1u2r3e4"
    host = "voipsetkl8.mzph8npyy2.tsdb.cloud.timescale.com"#"postgresql-152351-0.cloudclusters.net"
    port = "31768"#"19991"
    combined_data = pd.DataFrame()

    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
    cur = conn.cursor()



    for e in tqdm(stocks_list, desc="Fetching data", unit="stock", ncols=100):
        print(e)
        query = '''select * from nse_stocck_1min_cleaned where "Stock"=%s and "Date" >= %s and "Date" <= %s;'''
        cur.execute(query, (e,start_date,end_date))
        df = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
        combined_data = pd.concat([combined_data, df], ignore_index=True)

    conn.close()
    return combined_data.reset_index(drop=True)




# Program

In [5]:

# User Inputs
stock = input("Enter Name of Stock: ")
window_time  = int(input("Enter Window Time: "))

threshold = 2  # threshold value

#
industryStocks = pd.read_csv('Sectorlist-Nifty500.csv')

# Get a list of stocks from the same industry
industry_stock_list = get_same_industry_list(industryStocks, stock)

industry_stock_list.remove(stock)
industry_stock_list.append(stock)

if industry_stock_list is None:
    pass

start_date,end_date = data_start_last_date(stock)

# Get stock data for the list of stocks and reset index
combined_data = get_stock_data(industry_stock_list,start_date,end_date).reset_index(drop=True)
print(combined_data)

query_pattern_length = Query_pattern_length(window_time)
duration_in_minutes = int(f"{query_pattern_length * 5 if window_time >= 120 else query_pattern_length}")

# Get the index, raw index, selected pattern, time series modified, and end index
index, raw_index, selected_pattern, time_series_mod, end_index = query_pattern(combined_data, stock, query_pattern_length, window_time)

with open('selected_pattern.csv', 'a', newline='') as csvfile:
  csv_writer = csv.writer(csvfile)
  column_names = selected_pattern.columns.tolist()
  csv_writer.writerows([column_names])
  csv_writer.writerows(selected_pattern.values.tolist())

valid_match = True
col_name = 'Close'

# Find matches in the time series data
matches = find_matches(selected_pattern, time_series_mod, query_pattern_length, valid_match, col_name)

# Get matches and subsequent data frames
matches_dfs, subsequent_dfs = subsequent_df(combined_data, duration_in_minutes, matches, window_time, time_series_mod)

#Get the Snapshot of Matches
matches_concatenated = pd.concat([df.assign(key=i) for i, df in enumerate(matches_dfs)], ignore_index=True)
# Save the concatenated DataFrame to an Excel file
matches_concatenated.to_excel(f'matches_concatenated_{stock}.xlsx', index=False)
#Get the Subsequnet of Matches
# Concatenate all DataFrames in subsequent_dfs with a new 'key' column
subsequent_concatenated = pd.concat([df.assign(key=i) for i, df in enumerate(subsequent_dfs)], ignore_index=True)
# Save the concatenated DataFrame to an Excel file
subsequent_concatenated.to_excel(f'subsequent_concatenated_{stock}.xlsx', index=False)

# Determine buy or sell action based on matches and subsequent data frames
action_counts, bs_index = buyorsell(matches_dfs, subsequent_dfs, threshold)
max_action = majority_decision(action_counts,len(matches)/2)
#concatenated_df = pd.concat([matches_dfs[0], matches_dfs[1], matches_dfs[2]], axis=0)

# Prepare data for CSV output
candlestick = "5 m" if window_time >= 120 else "1 m"
Snapshot_period_start_datetime = f"{selected_pattern.iloc[0]['Date']}  {selected_pattern.iloc[0]['Time']}"
Snapshot_period_end_datetime = f"{selected_pattern.iloc[-1]['Date']}  {selected_pattern.iloc[-1]['Time']}"
min_euclidean_distance = "{:.8f}".format(matches.iloc[0][0])
max_euclidean_distance = "{:.8f}".format(matches.iloc[-1][0])

# Define column names for CSV output
column_names = ['Stock Name', 'Holding period', 'Query pattern length', 'Duration in minutes',
                'Snapshot period start datetime', 'Snapshot period end datetime',
                'Buy Count', 'Sell Count', 'Indecision Count',
                'Majority', 'Candlestick', 'Min Euclidean Distance', 'Max Euclidean Distance',
                'Risk Reward Ratio(Threshold)', 'Close Price']
# Append data to CSV file
data_to_append = [stock, str(window_time)+' m', query_pattern_length, duration_in_minutes,
                    Snapshot_period_start_datetime, Snapshot_period_end_datetime,
                    action_counts['Buy'], action_counts['Sell'], action_counts['Indecision'],
                    max_action, candlestick, min_euclidean_distance, max_euclidean_distance,
                    threshold,selected_pattern.iloc[-1]['Close']]


with open('count.csv', 'a', newline='') as csvfile:

  csv_writer = csv.writer(csvfile)

  csv_writer.writerow(column_names)

  csv_writer.writerow(data_to_append)


Enter Name of Stock: ADANIENT
Enter Window Time: 30
2024-03-12
<class 'str'> <class 'str'>


Fetching data:   0%|                                                       | 0/5 [00:00<?, ?stock/s]

AEGISCHEM


Fetching data:  20%|█████████▍                                     | 1/5 [00:04<00:17,  4.40s/stock]

SHARDACROP


...

MMTC


Fetching data:  80%|█████████████████████████████████████▌         | 4/5 [00:11<00:02,  2.51s/stock]

ADANIENT


Fetching data: 100%|███████████████████████████████████████████████| 5/5 [00:13<00:00,  2.75s/stock]


             Stock        Date      Time     Open     High      Low    Close  Volume
0        AEGISCHEM  2021-03-15  09:15:00   305.95   306.00   303.00   303.00    7278
1        AEGISCHEM  2021-03-15  09:16:00   303.55   303.60   302.65   303.60    3210
2        AEGISCHEM  2021-03-15  09:17:00   303.50   305.00   303.50   304.90    6439
3        AEGISCHEM  2021-03-15  09:18:00   305.00   306.00   305.00   305.00    3473
4        AEGISCHEM  2021-03-15  09:19:00   305.00   305.00   303.70   304.45     787
...            ...         ...       ...      ...      ...      ...      ...     ...
1237808   ADANIENT  2024-03-12  13:31:00  3164.95  3166.00  3163.05  3163.60     965
1237809   ADANIENT  2024-03-12  13:32:00  3162.85  3165.10  3162.85  3164.75     827
1237810   ADANIENT  2024-03-12  13:33:00  3165.00  3165.65  3163.75  3164.85    1098
1237811   ADANIENT  2024-03-12  13:34:00  3164.85  3165.10  3162.05  3162.10     751
1237812   ADANIENT  2024-03-12  13:35:00  3163.75  3163.75  3163.

In [ ]:
selected_pattern

,Stock,Date,Time,Open,High,Low,Close,Volume
3329209,SBIN,2024-03-12,12:16:00,763.65,763.65,763.50,763.60,12434
3329210,SBIN,2024-03-12,12:17:00,763.60,763.75,763.20,763.20,17905
3329211,SBIN,2024-03-12,12:18:00,763.20,763.45,762.60,762.70,31114
3329212,SBIN,2024-03-12,12:19:00,762.70,762.70,761.50,761.80,83004
3329213,SBIN,2024-03-12,12:20:00,761.85,761.95,761.05,761.05,63683
...,...,...,...,...,...,...,...,...
3329294,SBIN,2024-03-12,13:41:00,760.40,760.50,760.00,760.40,21750
3329295,SBIN,2024-03-12,13:42:00,760.40,760.75,760.25,760.65,19890
3329296,SBIN,2024-03-12,13:43:00,760.65,761.00,760.00,760.10,34520
3329297,SBIN,2024-03-12,13:44:00,760.00,760.60,760.00,760.20,27704
